![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# What is Context Engineering?

## Learning Objectives (25 minutes)
By the end of this notebook, you will be able to:
1. **Define** context engineering and explain its importance in AI systems
2. **Identify** the four core components of context engineering
3. **Compare** AI agents with and without context engineering using concrete examples
4. **Describe** the role of memory in intelligent agents
5. **Recognize** real-world applications and benefits of context engineering

## Prerequisites
- Basic understanding of AI and language models
- Familiarity with Python programming
- No prior experience with Redis or vector databases required

---

## Introduction

**Context Engineering** is the discipline of designing, implementing, and optimizing context management systems for AI agents and applications. It's the practice of ensuring that AI systems have the right information, at the right time, in the right format to make intelligent decisions and provide relevant responses.

Think of context engineering as the "memory and awareness system" for AI agents - it's what allows them to:
- Remember past conversations and experiences
- Understand their role and capabilities
- Access relevant information from large knowledge bases
- Maintain coherent, personalized interactions over time

## Why Context Engineering Matters

Without proper context engineering, AI agents are like people with severe amnesia - they can't remember what happened five minutes ago, don't know who they're talking to, and can't learn from experience. This leads to:

❌ **Poor User Experience**
- Repetitive conversations
- Lack of personalization
- Inconsistent responses

❌ **Inefficient Operations**
- Redundant processing
- Inability to build on previous work
- Lost context between sessions

❌ **Limited Capabilities**
- Can't handle complex, multi-step tasks
- No learning or adaptation
- Poor integration with existing systems

## Core Components of Context Engineering

Context engineering involves several key components working together:

### 1. **System Context**
What the AI should know about itself and its environment:
- Role and responsibilities
- Available tools and capabilities
- Operating constraints and guidelines
- Domain-specific knowledge

### 2. **Memory Management**
How information is stored, retrieved, and maintained:
- **Working memory**: Persistent storage focused on the current task, including conversation context and task-related data
- **Long-term memory**: Knowledge learned across sessions, such as user preferences and important facts

### 3. **Context Retrieval**
How relevant information is found and surfaced:
- Semantic search and similarity matching
- Relevance ranking and filtering
- Context window management

### 4. **Context Integration**
How different types of context are combined:
- Merging multiple information sources
- Resolving conflicts and inconsistencies
- Prioritizing information by importance

## Real-World Example: University Class Agent

Let's explore context engineering through a practical example - a university class recommendation agent. This agent helps students find courses, plan their academic journey, and provides personalized recommendations.

### Without Context Engineering
```
Student: "I'm interested in programming courses"
Agent: "Here are all programming courses: CS101, CS201, CS301..."

Student: "I prefer online courses"
Agent: "Here are all programming courses: CS101, CS201, CS301..."

Student: "What about my major requirements?"
Agent: "I don't know your major. Here are all programming courses..."
```

### With Context Engineering
```
Student: "I'm interested in programming courses"
Agent: "Great! I can help you find programming courses. Let me search our catalog...
        Based on your Computer Science major and beginner level, I recommend:
        - CS101: Intro to Programming (online, matches your preference)
        - CS102: Data Structures (hybrid option available)"

Student: "Tell me more about CS101"
Agent: "CS101 is perfect for you! It's:
        - Online format (your preference)
        - Beginner-friendly
        - Required for your CS major
        - No prerequisites needed
        - Taught by Prof. Smith (highly rated)"
```

### ✅ Knowledge Check: Context Engineering Basics

**Question 1**: What are the four core components of context engineering?
- [ ] System Context, Memory Management, Context Retrieval, Context Integration
- [ ] Prompts, Tools, Memory, Optimization
- [ ] Input, Processing, Output, Feedback
- [ ] Data, Models, APIs, Interfaces

**Question 2**: Which type of memory is session-scoped?
- [ ] Long-term memory
- [ ] Working memory
- [ ] Semantic memory
- [ ] Episodic memory

**Question 3**: What happens to an AI agent without context engineering?
- [ ] It becomes more efficient
- [ ] It loses memory between conversations
- [ ] It processes faster
- [ ] It uses fewer tokens

*Answers: 1-A, 2-B, 3-B*

## Environment Setup

Before we explore context engineering in action, let's set up our environment with the necessary dependencies and connections.

**Note**: For complete environment setup instructions, see the next notebook: `03_setup_environment.ipynb`

In [ ]:
# Install the Redis Context Course package
%pip install --upgrade -q -e ../../reference-agent

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up environment with consistent defaults
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
AGENT_MEMORY_URL = os.getenv("AGENT_MEMORY_URL", "http://localhost:8088")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Non-interactive check for OpenAI key
if not OPENAI_API_KEY:
    print("⚠️  OPENAI_API_KEY is not set. Some examples that call OpenAI will be skipped.")
    print("   See the setup notebook for configuration instructions.")
else:
    print("✅ Environment configured successfully")
    print(f"   Redis URL: {REDIS_URL}")
    print(f"   Agent Memory URL: {AGENT_MEMORY_URL}")

In [ ]:
# Import the Redis Context Course components
try:
    from redis_context_course.models import Course, StudentProfile, DifficultyLevel, CourseFormat
    from redis_context_course import MemoryClient
    from redis_context_course.course_manager import CourseManager
    from redis_context_course.redis_config import redis_config
    
    # Check Redis connection
    redis_available = redis_config.health_check()
    print(f"Redis connection: {'✅ Connected' if redis_available else '❌ Failed'}")
    print("✅ Redis Context Course package imported successfully")
    
except ImportError as e:
    print(f"⚠️  Import error: {e}")
    print("   Please ensure the reference agent is installed correctly.")
    redis_available = False

## Context Engineering in Action

Now that our environment is ready, let's explore the different types of context our agent manages:

### 1. System Context Example

System context defines what the agent knows about itself. This is typically provided as a system prompt:

In [ ]:
# Example of a system prompt - the agent's instructions and constraints
system_prompt = """
You are a helpful university class recommendation agent for Redis University.
Your role is to help students find courses, plan their academic journey, and
answer questions about the course catalog.

## Your Responsibilities

- Help students discover courses that match their interests and goals
- Provide accurate information about course content, prerequisites, and
  schedules
- Remember student preferences and use them to personalize recommendations
- Guide students toward courses that align with their major requirements

## Important Constraints

- Only recommend courses that exist in the course catalog (use the
  search_courses tool to verify)
- Always check prerequisites before recommending a course
- Respect student preferences for course format (online, in-person, hybrid)
- Be honest when you don't know something - don't make up course information
- If a student asks about a course that doesn't exist, help them find similar
  alternatives

## Interaction Guidelines

- Be friendly, encouraging, and supportive
- Ask clarifying questions when student requests are vague
- Explain your reasoning when making recommendations
- Keep responses concise but informative
- Use the student's name when you know it

## Tools Available

You have access to tools for searching the course catalog and managing student
memories. Use these tools to provide accurate, personalized recommendations.
"""

print("🤖 System Prompt Example:")
print("=" * 60)
print(system_prompt)
print("=" * 60)
print("\nThis system prompt will be included in every conversation turn,")
print("giving the LLM consistent instructions about its role and behavior.")

### 2. Student Context Example

Student context represents what the agent knows about the user:

In [ ]:
# Example student profile - user context
if redis_available:
    student = StudentProfile(
        name="Arsene Wenger",
        email="arsene.wenger@university.edu",
        major="Computer Science",
        year=2,
        completed_courses=["CS101", "MATH101", "ENG101"],
        current_courses=["CS201", "MATH201"],
        interests=["machine learning", "web development", "data science"],
        preferred_format=CourseFormat.ONLINE,
        preferred_difficulty=DifficultyLevel.INTERMEDIATE,
        max_credits_per_semester=15
    )
    
    print("👤 Student Context:")
    print(f"Name: {student.name}")
    print(f"Major: {student.major} (Year {student.year})")
    print(f"Completed: {len(student.completed_courses)} courses")
    print(f"Current: {len(student.current_courses)} courses")
    print(f"Interests: {', '.join(student.interests)}")
    print(f"Preferences: {student.preferred_format.value}, {student.preferred_difficulty.value} level")
else:
    print("⚠️  Skipping student profile example (Redis not available)")

### 3. Memory Context Example

Memory context includes past conversations and stored knowledge. Our agent uses the Agent Memory Server to store and retrieve memories.

**Note:** This requires the Agent Memory Server to be running. See Section 3 notebooks for detailed memory operations.

In [ ]:
# Memory demonstration (requires Agent Memory Server)
try:
    from agent_memory_client import MemoryAPIClient as MemoryClient, MemoryClientConfig
    from agent_memory_client.models import MemoryTypeEnum, ClientMemoryRecord
    
    # Initialize memory client
    config = MemoryClientConfig(
        base_url=AGENT_MEMORY_URL,
        default_namespace="redis_university"
    )
    memory_client = MemoryClient(config=config)
    
    # Example of storing different types of memories
    async def demonstrate_memory_context():
        try:
            await memory_client.create_long_term_memory([
                ClientMemoryRecord(
                    text="I prefer online courses because I work part-time",
                    memory_type=MemoryTypeEnum.SEMANTIC,
                    topics=["preferences", "schedule"]
                ),
                ClientMemoryRecord(
                    text="I want to specialize in machine learning and AI",
                    memory_type=MemoryTypeEnum.SEMANTIC,
                    topics=["goals", "career"]
                ),
                ClientMemoryRecord(
                    text="Student struggled with calculus but excelled in programming courses",
                    memory_type=MemoryTypeEnum.SEMANTIC,
                    topics=["academic_performance", "strengths"]
                )
            ])
            
            print("🧠 Memory Context Stored:")
            print("✅ Preference stored")
            print("✅ Goal stored")
            print("✅ Academic performance noted")
            
            # Retrieve relevant memories using semantic search
            results = await memory_client.search_long_term_memory(
                text="course recommendations for machine learning",
                namespace={"eq": "redis_university"},
                limit=3
            )
            
            print(f"\n🔍 Retrieved {len(results.memories)} relevant memories:")
            for memory in results.memories:
                print(f"  • [{memory.memory_type}] {memory.text[:60]}...")
                
        except Exception as e:
            print(f"⚠️  Memory server not available: {e}")
            print("   This is expected if Agent Memory Server is not running.")
    
    # Run the memory demonstration
    await demonstrate_memory_context()
    
except ImportError:
    print("⚠️  Agent Memory Client not available")
    print("   Memory examples will be covered in Section 3 notebooks.")

## Context Integration in Practice

Now let's see how all these context types work together to construct the actual prompt sent to the LLM:

In [ ]:
# Demonstrate how context sources are integrated into a complete prompt
def demonstrate_context_integration():
    """
    This demonstrates how we assemble different context sources into a complete prompt.
    """
    print("🎯 Context Integration: Building the Complete Prompt")
    print("=" * 70)

    # 1. Student asks for recommendations
    user_query = "What courses should I take next semester?"
    print(f"\n📝 User Query: '{user_query}'")

    # 2. Simulated memory retrieval (would normally come from Agent Memory Server)
    print("\n🔍 Step 1: Searching long-term memory...")
    simulated_memories = [
        "User prefers online courses due to work schedule",
        "User is interested in machine learning and AI",
        "User struggled with calculus but excelled in programming"
    ]
    memories_text = "\n".join([f"- {memory}" for memory in simulated_memories])
    print(f"   Found {len(simulated_memories)} relevant memories")

    # 3. Get student profile information
    print("\n👤 Step 2: Loading student profile...")
    if redis_available:
        student_context = f"""Name: {student.name}
Major: {student.major} (Year {student.year})
Completed Courses: {', '.join(student.completed_courses)}
Current Courses: {', '.join(student.current_courses)}
Interests: {', '.join(student.interests)}
Preferred Format: {student.preferred_format.value}
Preferred Difficulty: {student.preferred_difficulty.value}"""
    else:
        student_context = """Name: Sample Student
Major: Computer Science (Year 2)
Completed Courses: CS101, MATH101, ENG101
Current Courses: CS201, MATH201
Interests: machine learning, web development, data science
Preferred Format: online
Preferred Difficulty: intermediate"""
    
    print("   Profile loaded")

    # 4. Assemble the complete prompt
    print("\n🔧 Step 3: Assembling complete prompt...")

    # This is the actual prompt that would be sent to the LLM
    complete_prompt = f"""SYSTEM PROMPT:
{system_prompt}

STUDENT PROFILE:
{student_context}

POTENTIALLY RELEVANT MEMORIES:
{memories_text}

USER QUERY:
{user_query}

Please provide a helpful response based on the student's profile, memories, and query."""

    # 5. Display the assembled prompt
    print("\n" + "=" * 70)
    print("📋 COMPLETE ASSEMBLED PROMPT (sent to LLM):")
    print("=" * 70)
    print(complete_prompt)
    print("=" * 70)

    print("\n💡 Key Points:")
    print("   • System prompt defines the agent's role and constraints")
    print("   • Student profile provides current context about the user")
    print("   • Memories add relevant information from past conversations")
    print("   • User query is the current request")
    print("   • All assembled into a single prompt for the LLM")

demonstrate_context_integration()

## 🛠️ Hands-on Exercise: Compare Agent Behaviors

**Task**: Think about the examples we've shown and answer these questions:

1. **Without Context**: What problems would you encounter with an agent that has no memory?
2. **With Context**: How does context engineering improve the user experience?
3. **Real-World**: Can you think of AI systems you use that demonstrate good or poor context management?

**Expected Time**: 5 minutes  
**Deliverable**: Written reflection (3-5 sentences each)

### Your Answers:
*(Write your thoughts here or in a separate document)*

1. **Without Context**: 

2. **With Context**: 

3. **Real-World**: 

## Key Takeaways

From this introduction to context engineering, we can see several important principles:

### 1. **Context is Multi-Dimensional**
- **System context**: What the AI knows about itself
- **User context**: What the AI knows about the user
- **Domain context**: What the AI knows about the subject matter
- **Conversation context**: What has been discussed recently
- **Historical context**: What has been learned over time

Some of these sources are static, updated only when the agent's code changes,
while others may be retrieved dynamically from external sources, such as
via APIs or vector search.

### 2. **Memory is Essential**
- **Working memory**: Maintains conversation flow and task-related context
- **Long-term memory**: Enables learning and personalization across sessions

### 3. **Context Must Be Actionable**
- Information is only valuable if it can improve responses
- Context should be prioritized by relevance and importance -- this is often done through scoring and filtering
- The system must be able to integrate multiple context sources

### 4. **Context Engineering is Iterative**
- Systems improve as they gather more context -- though as we'll see in the course, there are limits
- Context quality affects response quality
- Feedback loops help refine context management

## Next Steps

In the next notebook, we'll explore the **Project Overview** - diving deeper into the Redis University Class Agent architecture and seeing how all these concepts come together in a real implementation.

After that, we'll cover **Environment Setup** to get you ready for hands-on work with the system.

## 🤔 Reflection: Real-World Applications

Think about AI systems you use daily (ChatGPT, virtual assistants, recommendation systems):

1. Which ones remember your preferences across sessions?
2. How does this memory affect your experience?
3. What would happen if they forgot everything each time?
4. Can you identify examples of good vs. poor context management?

**Consider sharing your thoughts in the discussion forum or with fellow learners.**

---

The power of context engineering lies in its ability to make AI systems more intelligent, personalized, and useful. As we'll see in the following notebooks, the technical implementation of these concepts using Redis, LangGraph, and modern AI tools makes it possible to build sophisticated, context-aware applications.